In [1]:
# import numpy as np
# import torch

# from utils.quantile import get_quantile_level_analytically

# for _ in range(1):
#     number_of_quantile_levels = 5
#     quantile_levels = torch.linspace(0.05, 0.95, number_of_quantile_levels).to(**dataset.tensor_parameters)

#     calibration_set_size = 1000
#     X_calibration, Y_calibration = dataset.sample_joint(n_points=calibration_set_size)
#     U_calibration = model.push_y_given_x(y=Y_calibration, x=X_calibration)
#     U_calibration_radii = U_calibration.norm(dim=-1).flatten()

#     radii_calibrated = torch.quantile(U_calibration_radii, quantile_levels, interpolation='nearest')
#     radii_true = get_quantile_level_analytically(quantile_levels, distribution="gaussian", dimension=Y_calibration.shape[1])

#     X, Y = dataset.sample_joint(n_points=10000)
#     U = model.push_y_given_x(y=Y, x=X)
#     U_radii = U.norm(dim=-1)

#     operator_coverage = torch.greater_equal(radii_true.unsqueeze(1), U_radii.unsqueeze(0)).float().mean(dim=-1)
#     conformalized_operator_coverage = torch.greater_equal(radii_calibrated.unsqueeze(1), U_radii.unsqueeze(0)).float().mean(dim=-1)

#     print(quantile_levels.shape, operator_coverage.shape)
#     print(quantile_levels.tolist())
#     print(operator_coverage.tolist())
#     print(conformalized_operator_coverage.tolist())
#     print("____")

In [1]:
from pushforward_operators import FastNonLinearVectorQuantileRegression
from datasets import ConvexBananaDataset
import torch

model = FastNonLinearVectorQuantileRegression(1, 5, 100, 1)
dataset = ConvexBananaDataset({})
X_dataset, Y_dataset = dataset.sample_joint(n_points=1000)
dataloader = torch.utils.data.DataLoader(
    dataset=torch.utils.data.TensorDataset(X_dataset, Y_dataset),
    **{}
)

_ = model.fit(dataloader)

/Users/vladimir.kondratyev/conditional_quantile_function/src/pushforward_operators/fast_non_linear_vector_quantile_regression/vqr/__init__.py:5: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution
/Users/vladimir.kondratyev/conditional_quantile_function/src/pushforward_operators/fast_non_linear_vector_quantile_regression/vqr/solvers/regularized_lse.py:192: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y_th = tensor(Y, **dtd)


In [6]:
import numpy as np

X_test = np.arange(0.5, 2.5, 0.1).reshape(-1, 1)
vqfs = model.vector_quantile_regression.vector_quantiles(X_test, refine=True)
# U = torch.rand(100, 2)

# pushforwards = torch.cat([vqf.evaluate(u=U.detach().numpy(force=True)) for vqf in vqfs])

In [ ]:
U = torch.rand(100, 2)
vqfs[0].evaluate(u=U.detach().numpy(force=True))

ValueError: u must be of shape (d,), got shape (100, 2)